# Transcribe isoforms to get RNA sequence


In [1]:
%load_ext autoreload
%autoreload 2

import itertools
import re

import gffutils
import numpy as np
import pandas as pd
import pybedtools
import six

v19db_filename = '/projects/ps-yeolab/genomes/hg19/gencode/v19/gencode.v19.annotation.gtf.db'
v19db = gffutils.FeatureDB(v19db_filename)

folder = '/projects/ps-yeolab/obotvinnik/singlecell_pnms'
csv_folder = '{}/csvs_for_paper/'.format(folder)
bed_folder = '{}/bed'.format(folder)

alt_exons_bedfile = '{}/exon2.bed'.format(bed_folder)
constitutive_bedfile = '{}/constitutive_exons.bed'.format(bed_folder)


splicing_feature_folder = '{}/splicing_feature_data'.format(csv_folder)
alternative_feature_folder = '{}/alternative'.format(splicing_feature_folder)
constitutive_feature_folder = '{}/constitutive'.format(splicing_feature_folder)



splicing_feature_data = pd.read_csv('{}/splicing_feature_data.csv'.format(csv_folder), index_col=0)
six.print_(splicing_feature_data.shape)
splicing_feature_data.head()

/home/obotvinnik/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:810: UserWarning: Found matplotlib configuration in ~/.matplotlib/. To conform with the XDG base directory standard, this configuration location has been deprecated on Linux, and the new location is now /home/obotvinnik/.config/matplotlib/. Please move your configuration there to ensure that matplotlib will continue to find it in the future.
  _get_xdg_config_dir())
/home/obotvinnik/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (11,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(46193, 25)


,criteria,criteria_additional,criteria_full,ensembl_id,exon1,exon1_length,exon2,exon2_length,exon3,exon3_length,...,junction12,junction13,junction23,junction24,junction34,strand,transcript_id,exon2_divisible_by_3,one_ensembl_id,splice_type
event_id,,,,,,,,,,,,,,,,,,,,,
exon:chr10:100190328-100190427:-@exon:chr10:100189548-100189646:-@exon:chr10:100189330-100189399:-,best,appris_principal,"best,appris_principal",ENSG00000107521,exon:chr10:100190328-100190427:-,100,exon:chr10:100189548-100189646:-,99,exon:chr10:100189330-100189399:-,70,...,chr10:100189647-100190327:-,chr10:100189400-100190327:-,chr10:100189400-100189547:-,NaN,NaN,-,"ENST00000361490.4,ENST00000414009.1,ENST000003...",True,ENSG00000107521,SE
exon:chr10:100193697-100193848:-@exon:chr10:100190888-100191048:-@exon:chr10:100190328-100190427:-,only one,NaN,only one,ENSG00000107521,exon:chr10:100193697-100193848:-,152,exon:chr10:100190888-100191048:-,161,exon:chr10:100190328-100190427:-,100,...,chr10:100191049-100193696:-,chr10:100190428-100193696:-,chr10:100190428-100190887:-,NaN,NaN,-,"ENST00000361490.4,ENST00000414009.1,ENST000003...",False,ENSG00000107521,SE
exon:chr10:100195392-100195529:-@exon:chr10:100195029-100195171:-@exon:chr10:100193697-100193848:-,best,appris_principal,"best,appris_principal",ENSG00000107521,exon:chr10:100195392-100195529:-,138,exon:chr10:100195029-100195171:-,143,exon:chr10:100193697-100193848:-,152,...,chr10:100195172-100195391:-,chr10:100193849-100195391:-,chr10:100193849-100195028:-,NaN,NaN,-,"ENST00000361490.4,ENST00000325103.6,ENST000004...",False,ENSG00000107521,SE
exon:chr10:101165513-101165617:-@exon:chr10:101163481-101163631:-@exon:chr10:101163226-101163391:-,only one,NaN,only one,ENSG00000120053,exon:chr10:101165513-101165617:-,105,exon:chr10:101163481-101163631:-,151,exon:chr10:101163226-101163391:-,166,...,chr10:101163632-101165512:-,chr10:101163392-101165512:-,chr10:101163392-101163480:-,NaN,NaN,-,"ENST00000543866.1,ENST00000370508.5",False,ENSG00000120053,SE
exon:chr10:101419263-101419345:+@exon:chr10:101419619-101419721:+@exon:chr10:101421203-101421385:+,only one,NaN,only one,ENSG00000198018,exon:chr10:101419263-101419345:+,83,exon:chr10:101419619-101419721:+,103,exon:chr10:101421203-101421385:+,183,...,chr10:101419346-101419618:+,chr10:101419346-101421202:+,chr10:101419722-101421202:+,NaN,NaN,+,ENST00000370489.4,False,ENSG00000198018,SE


In [2]:
figure_folder = '/home/obotvinnik/Dropbox/figures2/singlecell_pnms/isoform_rna_properties'
! mkdir $figure_folder

mkdir: cannot create directory `/home/obotvinnik/Dropbox/figures2/singlecell_pnms/isoform_rna_properties': File exists


## Transcribe isoforms

Get the RNA sequence form the DNA sequence

In [ ]:
prefix = 'isoform_transcriptions'
transcribed_fasta = '{}/{}.fa'.format(alternative_feature_folder, prefix)

In [ ]:
%%time

import itertools
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO
from Bio.Alphabet import generic_dna, generic_protein
import six

hg19_fasta = '/projects/ps-yeolab/genomes/hg19/gencode/v19/GRCh37.p13.genome.fa'

def overlap(x, y):
    return not ((x.start > y.stop) or (x.stop < y.start))

seqrecords = []


splice_type_isoform1_exons = {'SE': ('exon1', 'exon3'), 'MXE': ('exon1', 'exon3', 'exon4')}
splice_type_isoform2_exons = {'SE': ('exon1', 'exon2','exon3'), 'MXE': ('exon1', 'exon2', 'exon4')}

for i, (event_id, row) in enumerate(splicing_feature_data.iterrows()):
    if (i+1) % 1000 == 0:
        six.print_('\t Features completed', i+1)
        
    isoform1_exons = splice_type_isoform1_exons[row.splice_type]
    isoform2_exons = splice_type_isoform2_exons[row.splice_type]
    
    isoform_to_exons = {'isoform1': map(lambda x: v19db[row[x]], isoform1_exons),
                        'isoform2': map(lambda x: v19db[row[x]], isoform2_exons)}
        
    isoform_to_seq = {'isoform1': [], 'isoform2': []}

    reverse = isoform_to_exons['isoform1'][0].strand == '-'
    
    for isoform, exons in isoform_to_exons.items():
        if reverse:
            sequence = Seq(''.join(exon.sequence(hg19_fasta)[::-1] for exon in exons), generic_dna).complement()
        else:
            sequence = Seq(''.join(exon.sequence(hg19_fasta) for exon in exons), generic_dna)
        transcribed = sequence.transcribe()
        seqrecord = SeqRecord(transcribed, id='{0}|{1}'.format(event_id, isoform))
        seqrecords.append(seqrecord)


with open(transcribed_fasta, 'w') as f:
    SeqIO.write(seqrecords, f, 'fasta')

In [ ]:
! wc -l $transcribed_fasta

In [ ]:
! head $transcribed_fasta

In [ ]:
import seaborn as sns
%matplotlib inline

In [ ]:
sns.set(style='ticks', context='talk')

## Calculate miRNA hybridization

For RNA targets, James Broughton from the Pasquinelli lab recommends `RNAhybrid`, and to use just the first 17 nt of the mature miRNA. Wanted to use `fastx-trimmer` but it only takes DNA sequences  - `U`'s are illegal :(

In [ ]:
from Bio import SeqIO
import sys
import os

result_seq = []
filename = '/projects/ps-yeolab/genomes/mirbase/release_21/human_mature.fa'
with open(filename) as infile:
    for seq in SeqIO.parse(infile, 'fasta'):
        result_seq.append(seq[:17])

trimmed_filename = '/projects/ps-yeolab/genomes/mirbase/release_21/human_mature_17bp.fa'
with open(trimmed_filename, 'w') as outfile:
    SeqIO.write(result_seq, outfile, 'fasta')

Submit a compute job to calculate microRNA hybridization.

In [ ]:
from gscripts.qtools import Submitter

mirna_seqs = '/projects/ps-yeolab/genomes/mirbase/release_21/human_mature_17bp.fa'
rnahybrid_results = transcribed_fasta.replace('.fa', 'RNAhybrid_mirbase_human_mature_17bp.txt')
command = 'RNAhybrid -e -27 -c -s 3utr_human -q {} -t {} > {}'.format(mirna_seqs, transcribed_fasta, rnahybrid_results)
sub = Submitter([command], 'RNAhybrid', walltime='168:00:00', write_and_submit=True)

In [ ]:
ls -lha RNAhybrid*

In [15]:
!tail RNAhybrid*

==> RNAhybrid.sh <==
#PBS -V
#PBS -l walltime=168:00:00
#PBS -l nodes=1:ppn=1
#PBS -A yeo-group
#PBS -q home

# Go to the directory from which the script was called
cd $PBS_O_WORKDIR
RNAhybrid -e -27 -c -s 3utr_human -q /projects/ps-yeolab/genomes/mirbase/release_21/human_mature_17bp.fa -t /projects/ps-yeolab/obotvinnik/singlecell_pnms/skipped_exon_isoform_transcriptions.fa > /projects/ps-yeolab/obotvinnik/miso_helpers/hg19/se_exon2_RNAhybrid_mirbase_human_mature_17bp.txt


==> RNAhybrid.sh.err <==

==> RNAhybrid.sh.out <==
Nodes:        tscc-2-51


Reading the output takes a LONG time

In [10]:
%%time
import pandas as pd
from collections import Counter

rnahybrid = pd.read_csv('/projects/ps-yeolab/obotvinnik/miso_helpers/hg19/se_exon2_RNAhybrid_mirbase_human_mature_17bp.txt', 
                        sep=':', 
#                         index_col=[0, 1, 3], 
                        header=None,
                        names=['chrom', 'start-stop', 'exon_length', 'mirna', 'mirna_length',
                                                                 'minimum_free_energy', 'p_value', 'target_bound_start', 'mirna_unbound',
                                                                 'mirna_bound', 'exon_bound', 'exon_bound'])
rnahybrid = rnahybrid.ix[rnahybrid.minimum_free_energy < -28]
grouped = rnahybrid.groupby(['chrom', 'start-stop'])
chrom_startstop_mirna = grouped.mirna.apply(lambda x: ','.join('{}[{}]'.format(k, v) for k,v in Counter(x).items()))
chrom_startstop_mirna.head()

CPU times: user 13.6 s, sys: 991 ms, total: 14.6 s
Wall time: 15.3 s


In [16]:
! head /projects/ps-yeolab/obotvinnik/miso_helpers/hg19/se_exon2_RNAhybrid_mirbase_human_mature_17bp.txt

target too long: exon:chr10:102114184-102114389:+@exon:chr10:102116289-102116521:+@exon:chr10:102120491-102124591:+|isoform2
target too long: exon:chr10:102114184-102114389:+@exon:chr10:102116289-102116521:+@exon:chr10:102120491-102124591:+|isoform1
target too long: exon:chr10:103298050-103298128:+@exon:chr10:103310456-103310648:+@exon:chr10:103312912-103317078:+|isoform2
target too long: exon:chr10:103298050-103298128:+@exon:chr10:103310456-103310648:+@exon:chr10:103312912-103317078:+|isoform1
exon:chr10:104140263-104140443:+@exon:chr10:104140884-104141012:+@exon:chr10:104141813-104142656:+|isoform2:1154:hsa-let-7a-5p:17:-27.0:0.102090:837:C                C: UGCAGCCU CUGCCUC : AUGUUGGA GAUGGAG :         U       U
exon:chr10:104140263-104140443:+@exon:chr10:104140884-104141012:+@exon:chr10:104141813-104142656:+|isoform1:1025:hsa-let-7a-5p:17:-27.0:0.086480:708:C                C: UGCAGCCU CUGCCUC : AUGUUGGA GAUGGAG :         U       U
target too long: exon:chr10:106037652-106037874:+@